# CHAPTER 16. Metric-Predicted Variable on One or Two Groups

# Contents

* 16.1. Estimating the Mean and Standard Deviation of a Normal Distribution
* 16.2. Outliers and Robust Estimation : The t Distribution
* 16.3. Two Groups
* 16.4. Other Noise Distributions and Transforming Data
* 16.5. EXERCISES

<img src="figures/tbl15.3.png" width=600 />

In this chapter, we consider a situation in which we have a <font color="red">metric-predicted variable</font> that is observed for items <font color="blue">from one or two groups</font>.

* For example, 
    - we could measure 
        - the <font color="red">blood pressure</font> (i.e., a metric variable) for people 
        - randomly sampled <font color="blue">from first-year university students</font> (i.e., a single group).
    - In this case, we might be interested in 
        - how much the <font color="orange">group’s typical blood pressure differs</font> 
        - from the recommended value for people of that age as published by a federal agency.

* As another example, 
    - we could measure 
        - the <font color="red">IQ</font> (i.e., a metric variable) of people 
        - randomly sampled from everyone self-described as <font color="blue">vegetarian</font> (i.e., a single group). 
    - In this case, we could be interested in 
        - how much this <font color="orange">group’s IQ differs</font> 
        - from the general population’s average IQ of 100.

<img src="figures/tbl15.1.png" width=600 />

* In the context of the generalized linear model (GLM) introduced in the previous chapter, this chapter’s situation involves the most trivial cases of the linear core of the GLM, as indicated in the left cells of Table 15.1 (p. 434), with a link function that is the identity along with a normal distribution for describing noise in the data, as indicated in the first row of Table 15.2 (p. 443). 

<img src="figures/tbl15.2.png" width=600 />

* We will explore options for the prior distribution on parameters of the normal distribution, and methods for Bayesian estimation of the parameters. 
* We will also consider alternative noise distributions for describing data that have outliers.

# 16.1. Estimating the Mean and Standard Deviation of a Normal Distribution

* 16.1.1 Solution by mathematical analysis
* 16.1.2 Approximation by MCMC in JAGS

The normal distribution specifies the probability density of a value y, given the values of two
parameters, the mean μ and standard deviation σ :

<img src="figures/eq16.1.png" width=600 />

* To get an intuition for the <font color="red">normal distribution as a likelihood function</font>, consider three data values y1 = 85, y2 = 100, and y3 = 115, which are plotted as large dots in Figure 16.1. 

* Figure 16.1 <font color="red">shows p(D|μ, σ ) for different values of μ and σ</font> . As you can see, there are values of μ and σ that make the data most probable, but other nearby values also accommodate the data reasonably well

<img src="figures/fig16.1.png" width=600 />

<font color="red">The question is, given the data, how should we allocate credibility to combinations of μ and σ?</font>

* likelhood 
    - Figure 16.1 shows examples of p(D|μ, σ ) for a particular data set at different values of μ and σ 
* prior 
    -The prior, p(μ,σ), specifies the credibility of each combination of μ,σ values in the two-dimensional joint parameter space, without the data.
*  posterior
    - Bayes’ rule says that the posterior credibility of each combination of μ, σ values is the prior credibility times the likelihood, normalized by the marginal likelihood.

<img src="figures/eq16.2.png" width=600 />

<font color="red">Our goal now is to evaluate Equation 16.2 for reasonable choices of the prior distribution, p(μ, σ ).</font>

## 16.1.1 Solution by mathematical analysis

* We take a short algebraic tour before moving on to MCMC implementations.

### When σ is fixed, 

* It is convenient first to consider the case in which the standard deviation of the likelihood function is fixed at a specific value. In other words, <font color="red">the prior distribution on σ is a spike over that specific value</font>. We’ll denote that <font color="red">fixed value as σ = Sy</font>.  
* With this simplifying assumption, <font color="red">we are only estimating μ</font> because we are <font color="red">assuming perfectly certain prior knowledge about σ</font> .
* proir : When σ is fixed, then the <font color="red">prior distribution on μ</font> in Equation 16.2 can be easily chosen to be <font color="red">conjugate to the normal likelihood</font>.
    - The term “conjugate prior” was defined in Section 6.2, p. 126.
    - It turns out that the product of normal distributions is again a normal distribution; 
    - in other words, if the prior on μ is normal, then the posterior on μ is normal.
    - <font color="red">Let the prior distribution on μ be normal with mean Mμ and standard deviation Sμ</font> .
* likelihood * prior : 

<img src="figures/eq16.3.png" width=600 />

<img src="figures/eq16.4.png" width=600 />

<img src="figures/cap16.3.png" width=600 />

##### precision & posterior precision

<img src="figures/cap16.4.png" width=600 />

<img src="figures/eq16.5.png" width=600 />

* <font color="red">Thus, the posterior precision is the sum of the prior precision and the likelihood precision.</font>

##### posterior mean

<img src="figures/cap16.5.png" width=600 />

<img src="figures/eq16.6.png" width=600 />

* <font color="red">In other words, the posterior mean is a weighted average of the prior mean and the datum, with the weighting corresponding to the relative precisions of the prior and the likelihood.</font>

#### N value case

<img src="figures/cap16.6.png" width=600 />

##### posterior mean

<img src="figures/cap16.1.png" width=600 />

##### posterior precision

<img src="figures/cap16.2.png" />

* Notice that as the sample size N increases, the posterior mean is dominated by the data mean.


### When μ is fixed, 

* We can instead <font color="red">estimate the σ parameter</font> when μ is fixed. 
* It turns out that when μ is fixed, a <font color="red">conjugate prior for the precision</font> is <font color="red">the gamma distribution</font> (e.g., Gelman et al., 2013, p. 43). 
    - It is important to <font color="red">understand the meaning of a gamma prior on precision</font>. 
    - Consider a gamma distribution 
        - that is loaded <font color="orange">heavily over very small values, but has a long shallow tail extending over large values</font>. 
            - This sort of  gamma distribution on precision indicates that <font color="orange">we believe most strongly in small precisions</font>, but we <font color="orange">admit that large precisions are possible</font>. 
            - If this is a <font color="orange">belief about the precision of a normal likelihood function</font>, then this sort of <font color="orange">gamma distribution expresses a belief that the data will be more spread out</font>, because <font color="orange">small precisions imply large standard deviations</font>. 
        -  If the gamma distribution is instead loaded over <font color="blue">large values of precision</font>, 
            - it expresses a <font color="blue">belief that the data will be tightly clustered</font>.

##### 참고: gamma distribution ?

* https://en.wikipedia.org/wiki/Gamma_distribution

<img src="https://upload.wikimedia.org/wikipedia/commons/f/fc/Gamma_distribution_pdf.png" width=600 />

##### conjugate priors & gamma distiribution & precision

* Because of its role in <font color="red">conjugate priors for the normal likelihood function</font>, the <font color="red">gamma distribution is routinely used</font> as a <font color="red">prior on precision</font>. 
* But there is no logical necessity to do so, and <font color="red">modern MCMC methods permit more flexible</font> specification of priors. 
* Indeed, <font color="red">because precision is less intuitive than standard deviation</font>, it can be <font color="red">more useful instead to give the standard deviation a uniform prior that spans a wide range</font>.

### Summary

We have assumed that the data are generated by a normal likelihood function,
parameterized by a mean μ and standard deviation σ , and denoted y ∼ normal(y|μ, σ ).
For purposes of mathematical derivation, we made unrealistic assumptions that the prior
distribution is either a spike on σ or a spike on μ, in order to make three main
points:


1. <font color="red">A natural way to express a prior on μ</font> is with a <font color="red">normal distribution</font>, because this is <font color="red">conjugate</font> with the <font color="red">normal likelihood</font> when its <font color="red">standard deviation is fixed</font>.
2. <font color="red">A way to express a prior on the precision</font> 1/σ 2 is with a <font color="red">gamma distribution</font>, because this is <font color="red">conjugate with the normal likelihood when its mean is fixed</font>. However <font color="blue">in practice</font> the <font color="blue">standard deviation</font> can instead be <font color="blue">given a uniform prior</font> (or anything else that reflects prior beliefs, of course).
3. The <font color="red">formulas for Bayesian updating</font> of the parameter distribution are <font color="red">more conveniently expressed</font> in terms of <font color="red">precision than standard deviation</font>. <font color="red">Normal distributions are described</font> <font color="orange">sometimes</font> in terms of <font color="orange">standard deviation</font> and <font color="green">sometimes</font> in terms of <font color="green">precision</font>, so it is important to glean from context which is being referred to. <font color="red">In R and Stan</font>, the <font color="red">normal distribution</font> is parameterized <font color="red">by mean and standard deviation</font>. <font color="blue">In JAGS and BUGS</font>, the normal distribution is parameterized <font color="blue">by mean and precision</font>.

## 16.1.2 Approximation by MCMC in JAGS

It is easy to estimate the mean and standard deviation in JAGS.

* The right panel of Figure 16.2 instead puts a broad uniform distribution directly
on σ . The low and high values of the uniform distribution are set to be far outside any realistic
value for the data, so that the prior has minimal influence on the posterior. 
* The uniform prior on σ is easier to intuit than a gamma prior on precision, but the priors are not equivalent.

<img src="figures/fig16.2.png" width=600 />

In [ ]:
dataList = list(
    y = y ,
    Ntotal = length(y) ,
    meanY = mean(y) ,
    sdY = sd(y)
)

In [ ]:
model {
    for ( i in 1:Ntotal ) {
        y[i]  ̃ dnorm( mu , 1/sigmaˆ2 ) # JAGS uses precision
    }
    mu  ̃ dnorm( meanY , 1/(100*sdY)ˆ2 ) # JAGS uses precision
    sigma  ̃ dunif( sdY/1000 , sdY*1000 )
}

* For purposes of illustration, we use fictitious data.
* The data are 
    - <font color="red">IQ</font> (intelligence quotient) scores 
        - from <font color="red">a group</font> of people who have consumed a <font color="red">“smart drug.”</font> 
    - We know that IQ tests have been normed to the general population so that they have an <font color="blue">average score of 100 and a standard deviation of 15</font>. 
* Therefore, we would like to know <font color="red">how differently the smart-drug group</font> has performed relative to the <font color="red">general population average</font>.

<img src="figures/fig16.3.png" width=600 />

#### Jags-Ymet-Xnom1grp-Mnormal-Example.R

In [3]:
# 책에는 없는 코드. 실습을 위해 data 폴더에 넣은 코드를 실행시키기 위한 작업
cur_dir = getwd()
setwd(sprintf("%s/%s", cur_dir, 'data'))

In [4]:
# Example for Jags-Ymet-Xnom1grp-Mnormal.R 
#------------------------------------------------------------------------------- 
# Optional generic preliminaries:
#graphics.off() # This closes all of R's graphics windows.
#rm(list=ls())  # Careful! This clears all of R's memory!

In [5]:
#------------------------------------------------------------------------------- 
# Load The data file 
myDataFrame = read.csv( file="TwoGroupIQ.csv" )

In [7]:
head(myDataFrame)

,Score,Group
1,102,Smart Drug
2,107,Smart Drug
3,92,Smart Drug
4,101,Smart Drug
5,110,Smart Drug
6,68,Smart Drug


In [8]:
summary(myDataFrame)

     Score               Group   
 Min.   : 50.00   Placebo   :57  
 1st Qu.: 92.75   Smart Drug:63  
 Median :102.00                  
 Mean   :104.13                  
 3rd Qu.:112.00                  
 Max.   :208.00                  

In [9]:
# For purposes of this one-group example, use data from Smart Drug group:
myData = myDataFrame$Score[myDataFrame$Group=="Smart Drug"]

In [10]:
#------------------------------------------------------------------------------- 
# Optional: Specify filename root and graphical format for saving output.
# Otherwise specify as NULL or leave saveName and saveType arguments 
# out of function calls.
fileNameRoot = "OneGroupIQnormal-" 
graphFileType = "png"  #"eps" 

In [11]:
#------------------------------------------------------------------------------- 
# Load the relevant model into R's working memory:
source("Jags-Ymet-Xnom1grp-Mnormal.R")


*********************************************************************
Kruschke, J. K. (2015). Doing Bayesian Data Analysis, Second Edition:
A Tutorial with R, JAGS, and Stan. Academic Press / Elsevier.
*********************************************************************



Installing packages into ‘/home/moosung/R/x86_64-pc-linux-gnu-library/3.2’
(as ‘lib’ is unspecified)


ERROR: Error in contrib.url(repos, type): trying to use CRAN without setting a mirror


In [ ]:
#------------------------------------------------------------------------------- 
# Generate the MCMC chain:
mcmcCoda = genMCMC( data=myData , numSavedSteps=20000 , saveName=fileNameRoot )

In [ ]:
#------------------------------------------------------------------------------- 
# Display diagnostics of chain, for specified parameters:
parameterNames = varnames(mcmcCoda) # get all parameter names
for ( parName in parameterNames ) {
  diagMCMC( codaObject=mcmcCoda , parName=parName , 
            saveName=fileNameRoot , saveType=graphFileType )
}

In [ ]:
# 창으로 뜨는 것들을 없앤다.
graphics.off()

In [ ]:
#------------------------------------------------------------------------------- 
# Get summary statistics of chain:
summaryInfo = smryMCMC( mcmcCoda , 
                        compValMu=100.0 , ropeMu=c(99.0,101.0) ,
                        compValSigma=15.0 , ropeSigma=c(14,16) ,
                        compValEff=0.0 , ropeEff=c(-0.1,0.1) ,
                        saveName=fileNameRoot )
show(summaryInfo)

In [ ]:
# Display posterior information:
plotMCMC( mcmcCoda , data=myData , 
          compValMu=100.0 , ropeMu=c(99.0,101.0) ,
          compValSigma=15.0 , ropeSigma=c(14,16) ,
          compValEff=0.0 , ropeEff=c(-0.1,0.1) ,
          pairsPlot=TRUE , showCurve=FALSE ,
          saveName=fileNameRoot , saveType=graphFileType )

In [ ]:
graphics.off()

# 16.2. Outliers and Robust Estimation : The t Distribution

* 16.2.1 Using the t distribution in JAGS
* 16.2.2 Using the t distribution in Stan

<img src="figures/fig16.4.png" width=600 />

<img src="figures/fig16.5.png" width=600 />

<img src="figures/fig16.6.png" width=600 />

## 16.2.1 Using the t distribution in JAGS

<img src="figures/fig16.7.png" width=600 />

<img src="figures/fig16.8.png" width=600 />

## 16.2.2 Using the t distribution in Stan

<img src="figures/fig16.9.png" width=600 />

<img src="figures/fig16.10-1.png" width=600 />

<img src="figures/fig16.10-2.png" width=600 />

# 16.3. Two Groups

* 16.3.1 Analysis by NHST

<img src="figures/fig16.11.png" width=600 />

## 16.3.1 Analysis by NHST

<img src="figures/fig16.12.png" width=600 />

# 16.4. Other Noise Distributions and Transforming Data

# 16.5. EXERCISES

### Exercise 16.1.

#### [Purpose: Practice using different data files in the high-level script, with an interesting real example about alcohol preference of sexually frustrated males.]

<img src="figures/fig16.13.png" width=600 />

# 참고자료